In [23]:
import os
from typing import List
import pandas as pd
import datasets

from transformers import AutoTokenizer, AutoModelForMaskedLM

In [24]:
DATA_DIR = 'crisis_data'
FILE_BLACKLIST = [
    'crisis_data/Afera Rywina.xlsx',
    'crisis_data/Ministerstwo Zdrowia_respiratory od handlarza bronią.xlsx',
    'crisis_data/Fake news_baza publikacji.xlsx'
]

files = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f[-5:] == '.xlsx']
for f in FILE_BLACKLIST:
    files.remove(f)

In [33]:
def prepare_dataset(filenames: List[str]):
    dss = []
    for filename in filenames:
        df = pd.read_excel(filename)
        df = df[['Kryzys', 'Tytuł publikacji', 'Lead', 'Kontekst publikacji']]
        df['text'] = df.apply(lambda x: ".".join([x['Tytuł publikacji'], x['Lead'], x['Kontekst publikacji']]), axis=1)
        df = df[['Kryzys', 'text']]
        dss.append(datasets.Dataset.from_pandas(df))
    return datasets.concatenate_datasets(dss)

In [34]:
prepare_dataset(files)

TypeError: sequence item 1: expected str instance, float found

In [8]:
tokenizer = AutoTokenizer.from_pretrained("sdadas/polish-distilroberta")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

In [3]:
df = pd.read_excel(files[0])

In [22]:
df.columns

Index(['Id nowe', 'Id stare', 'Kryzys', 'Tytuł publikacji', 'Źródło', 'Autor',
       'Data wydania', 'Data i godzina dodania', 'Strona (prasa)',
       'Godzina publikacji (Radio, TV, Social media)', 'Czas trwania (RTV)',
       'Folder', 'Projekt', 'Rodzaj projektu', 'Udział', 'Rozmiar', 'Wydźwięk',
       'Ave szacunkowe (PLN)', 'Ave zweryfikowane (PLN)', 'Ave mnożone (PLN)',
       'Dotarcie (kontaktów)',
       'Zasięg (egz.), (słuchaczy), (widzów), (UU), (subskrybentów), (obserwujących)',
       'Wpływ', 'Typ medium', 'Tematyka', 'Social media - typ wpisu',
       'Czestotliwość (prasa)', 'Wydawca', 'Kraj', 'Obszar',
       'Obszar - szczegóły', 'Powierzchnia gazety', 'Id medium',
       'Podgrupa medium', 'Powierzchnia (Prasa, Internet)', 'Status',
       'Komunikaty', 'Tagi', 'Lead', 'Kontekst publikacji',
       'Język publikacji (ISO)', 'Link do publikacji',
       'Link źródłowy (internet)'],
      dtype='object')

In [16]:
X = df['Tytuł publikacji'].apply(lambda x: tokenizer(x, truncation=True))

In [17]:
X

0      [input_ids, attention_mask]
1      [input_ids, attention_mask]
2      [input_ids, attention_mask]
3      [input_ids, attention_mask]
4      [input_ids, attention_mask]
                  ...             
262    [input_ids, attention_mask]
263    [input_ids, attention_mask]
264    [input_ids, attention_mask]
265    [input_ids, attention_mask]
266    [input_ids, attention_mask]
Name: Tytuł publikacji, Length: 267, dtype: object

In [ ]:




model = AutoModelForMaskedLM.from_pretrained("sdadas/polish-distilroberta")